In [1]:
import pandas as pd
from os import path
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pickle
import numpy as np
import os
import json
import csv

In [2]:
DATA_DIR = "../../data"

In [3]:
chen_train = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train.csv"), index_col=0)
chen_test = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test.csv"), index_col=0)
print(len(chen_train))
print(len(chen_test))

1291
260


In [4]:
tap = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))

In [5]:
best_models = pd.read_csv(path.join(DATA_DIR, "evaluations/best_total.csv"))
model_configs = json.load(open(path.join(DATA_DIR, "evaluations/best_by_model_configs.json"), "r"))
data_configs = json.load(open(path.join(DATA_DIR, "evaluations/best_by_data_configs.json"), "r"))

In [6]:
def knn(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"kNN_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    #n_neighbors = int(parameters["n_neighbors"])
    model = KNeighborsClassifier(n_neighbors=parameters["n_neighbors"]) 
    return model, parameters, "kNN"

def logistic_regression(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"logistic_regression_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    #C = float(parameters["C"])
    lr = LogisticRegression(
        class_weight='balanced', max_iter=1000, random_state=42,
        C=parameters["C"], penalty=parameters["penalty"], solver=parameters["solver"]
    )
    return lr, parameters, "logistic_regression"

def random_forest(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"random_forest_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    rf = RandomForestClassifier(
        random_state=42, n_jobs=-1, class_weight='balanced', n_estimators=int(parameters["n_estimators"]),
        max_depth=int(parameters["max_depth"]), max_features=float(parameters["max_features"])
    )
    return rf, parameters, "random_forest"

def multilayer_perceptron(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"multilayer_perceptron_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    mlp = MLPClassifier(
        random_state=42, max_iter=int(1000), hidden_layer_sizes=parameters["hidden_layer_sizes"],
        activation=parameters["activation"]
    )
    return mlp, parameters, "multilayer_perceptron"

def svm(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"SVM_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    svc = SVC(
        max_iter=8000, probability=True, class_weight='balanced', C=parameters["C"],
        kernel=parameters["kernel"], gamma=parameters["gamma"]
    )
    return svc, parameters, "SVM"

def gradient_boosting(preprocessing, data_name, hp_dir):
    filename = path.join(hp_dir, f"gradient_boosting_{data_name}_{preprocessing}.json")
    parameters = json.load(open(filename))
    gb = GradientBoostingClassifier(
        random_state=42, n_iter_no_change=70, learning_rate=float(parameters["learning_rate"]),
        n_estimators=int(parameters["n_estimators"]), max_depth=int(parameters["max_depth"]),
        max_features=float(parameters["max_features"])
    )
    return gb, parameters, "gradient_boosting"

In [19]:
def integer_encoded(train_df, test_df):
    x_chen = pd.read_csv(path.join(DATA_DIR, "chen/integer_encoding/chen_integer_encoded.csv"), index_col=0)
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_csv(path.join(DATA_DIR, "tap/integer_encoding/tap_integer_encoded.csv"))
    x_tap.drop("Ab_ID", axis=1, inplace=True)
    return x_chen_train, x_chen_test, x_tap


def pybiomed(train_df, test_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/pybiomed/X_data.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_feather(path.join(DATA_DIR, "tap/pybiomed/X_TAP_data.ftr"))
    return x_chen_train, x_chen_test, x_tap


def protparam(train_df, test_df):
    x_chen = pd.read_csv(path.join(DATA_DIR, "chen/protparam/protparam_features.csv"))
    x_chen.rename({"Unnamed: 0": "Ab_ID"}, axis=1, inplace=True)
    x_chen = x_chen.drop("name", axis=1)
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    
    x_tap = pd.read_csv(path.join(DATA_DIR, "tap/protparam/protparam_features_tap.csv"))
    x_tap = x_tap.drop("Unnamed: 0", axis=1)
    return x_chen_train, x_chen_test, x_tap


def bert(train_df, test_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/bert/bert_chen_embeddings.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_feather(path.join(DATA_DIR, "tap/embeddings/bert/bert_tap_embeddings.ftr"))
    x_tap = x_tap.drop("Ab_ID", axis=1)
    return x_chen_train, x_chen_test, x_tap


def seqvec(train_df, test_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/embeddings/seqvec/seqvec_chen_embeddings.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_csv(path.join(DATA_DIR, "tap/embeddings/seqvec/seqvec_tap_embeddings.csv"), index_col=0)
    x_tap = x_tap.drop("Ab_ID", axis=1)
    return x_chen_train, x_chen_test, x_tap


def sapiens(train_df, test_df):
    x_chen = pd.read_csv(path.join(DATA_DIR, "chen/embeddings/sapiens/sapiens_chen_embeddings.csv"), index_col=0).drop("Y", axis=1)
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_csv(path.join(DATA_DIR, "tap/embeddings/sapiens/sapiens_tap_embeddings.csv"), index_col=0)
    x_tap = x_tap.drop(["Ab_ID", "Y"], axis=1)
    return x_chen_train, x_chen_test, x_tap


def onehot(train_df, test_df):
    x_chen = pd.read_feather(path.join(DATA_DIR, "chen/onehot/chen_onehot_short.ftr"))
    x_chen_train = x_chen.merge(train_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_chen_test = x_chen.merge(test_df[["Antibody_ID", "Y"]].reset_index(), left_on="Ab_ID", right_on="Antibody_ID").set_index('index').drop("Antibody_ID", axis=1)
    x_tap = pd.read_feather(path.join(DATA_DIR, "tap/onehot/tap_onehot_short.ftr"))
    x_tap = x_tap.drop(["Ab_ID"], axis=1)
    return x_chen_train, x_chen_test, x_tap

In [8]:
def scaling(train_df, test_df, tap_df):
    scaler = StandardScaler()
    scaler.fit(train_df.drop(["Ab_ID", "Y"], axis=1))
    x_train_tr = scaler.transform(train_df.drop(["Ab_ID", "Y"], axis=1))
    x_train_df = pd.DataFrame(data=train_df,  index=train_df.index, columns=train_df.drop(["Ab_ID", "Y"], axis=1).columns)
    x_train_df["Ab_ID"] = train_df["Ab_ID"]
    
    x_test_tr = scaler.transform(test_df.drop(["Ab_ID", "Y"], axis=1))
    x_test_df = pd.DataFrame(data=test_df,  index=test_df.index, columns=test_df.drop(["Ab_ID", "Y"], axis=1).columns)
    x_test_df["Y"] = test_df["Y"]
    x_test_df["Ab_ID"] = test_df["Ab_ID"]
    
    x_tap_tr = scaler.transform(tap_df)
    x_tap_df = pd.DataFrame(data=tap_df,  index=tap_df.index, columns=tap_df.columns)

    return x_train_df, train_df["Y"], x_test_df, x_tap_df

In [9]:
def output_evaluation(model_type, metrics, data, outpath, preprocessing):
    filename_sum = os.path.join(DATA_DIR, f"evaluations/{outpath}/all.csv")
    line = [model_type, data, preprocessing, metrics["f1"], metrics["mcc"], metrics["acc"],metrics["precision"],metrics["recall"],metrics["auc"]]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)


def train_and_eval(model_name, classifier, X_train, y_train, X_valid, y_valid,
                   data_name, outpath, preprocessing):
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_valid)
    filename = path.join(DATA_DIR, "evaluations", outpath, "models", f"{model_name}_{data_name}_{preprocessing}.pkl")
    with open(filename, 'wb') as f:
        pickle.dump(classifier, f)
    filename = path.join(DATA_DIR, "evaluations", outpath, f"{model_name}_{data_name}_{preprocessing}.csv")
    str_preds = [str(int(pred)) for pred in y_pred]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")
    metric_dict = {
        "f1": float(metrics.f1_score(y_valid, y_pred)),
        "acc": float(metrics.accuracy_score(y_valid, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_valid, y_pred)),
        "auc": float(metrics.roc_auc_score(y_valid, y_pred)),
        "precision": float(metrics.precision_score(y_valid, y_pred)),
        "recall": float(metrics.recall_score(y_valid, y_pred))
    }
    
    output_evaluation(model_name, metric_dict, data_name, outpath, preprocessing)

In [10]:
def test_on_tap(model_name, x_test, y_test,
                   data_name, outpath, preprocessing):
    filename = path.join(DATA_DIR, "evaluations", outpath, "models", f"{model_name}_{data_name}_{preprocessing}.pkl")
    with open(filename, 'rb') as f:
        estimator = pickle.load(f)
    y_pred = estimator.predict(x_test)
    metric_dict = {
        "f1": float(metrics.f1_score(y_test, y_pred)),
        "acc": float(metrics.accuracy_score(y_test, y_pred)),
        "mcc": float(metrics.matthews_corrcoef(y_test, y_pred)),
        "auc": float(metrics.roc_auc_score(y_test, y_pred)),
        "precision": float(metrics.precision_score(y_test, y_pred)),
        "recall": float(metrics.recall_score(y_test, y_pred))
    }
    filename_sum = os.path.join(DATA_DIR, f"evaluations/{outpath}/tap.csv")
    line = [model_name, data_name, preprocessing, metric_dict["f1"], metric_dict["mcc"], metric_dict["acc"],metric_dict["precision"],metric_dict["recall"],metric_dict["auc"], filename]
    with open(filename_sum, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter='\t')
        csvwriter.writerow(line)
    filename = path.join(DATA_DIR, "evaluations", outpath, f"{model_name}_{data_name}_{preprocessing}_tap.csv")
    str_preds = [str(int(pred)) for pred in y_pred]
    with open(filename, "wt") as f:
        f.write(",".join(str_preds) + "\n")

In [11]:
to_train = [
    (logistic_regression, bert),
    (logistic_regression, seqvec),
    (logistic_regression, pybiomed),
    (svm, bert),
    (svm, seqvec),
    (knn, seqvec),
    (gradient_boosting, protparam),
    (multilayer_perceptron, bert),
    (random_forest, seqvec),
    (logistic_regression, onehot),
    (logistic_regression, sapiens),
    (random_forest, integer_encoded),
    (random_forest, protparam)
]

In [22]:
def train_final(train_df, test_df, eval_dir, tap_data):
    for model_creator, data_rep in to_train:
        data_name = data_rep.__name__
        x_train, x_test, x_tap = data_rep(train_df, test_df)
        for prepro in [scaling]:
            prepro_name = prepro.__name__
            x_train_tr, y_train_tr, x_test_tr, tap_tr = prepro(x_train, x_test, x_tap)
            
            classifier, params, model_label = model_creator(prepro_name, data_name, path.join(DATA_DIR, "evaluations/hyperparameters"))
            train_and_eval(
                model_label, classifier, x_train_tr.drop(["Ab_ID"], axis=1), 
                y_train_tr, x_test_tr.drop(["Ab_ID", "Y"], axis=1), x_test_tr["Y"], 
                data_name, eval_dir, prepro_name
            )
            test_on_tap(
                model_label, tap_tr, tap_data["Y"], data_name, eval_dir, prepro_name
            )

In [13]:
eval_dir = "final"

In [23]:
train_final(chen_train, chen_test, eval_dir, tap)

/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi